In [1]:
import pandas as pd
import requests
import re
import os
from tqdm import tqdm

In [2]:
# import tarfile
# # Specify the path to your .tar file
# tar_path = "IGWN-GWOSC-snapshot-2023-11-04.tar"
# 
# # Open and list the contents of the .tar file
# with tarfile.open(tar_path, 'r') as tar:
#     file_names = tar.getnames()
#     print("Files in tar archive:", file_names)
# 
#     # Extract all files or a specific file
#     tar.extractall()  # Extracts to the current directory

In [3]:
filepath = "snapshot-2023-11-04/GWTC/GWTC.json"
df = pd.read_json(filepath)
events = list(df.index)
for i in range(len(events)):print(f"{i}. {events[i]}")

0. GW200322_091133-v1
1. GW200316_215756-v1
2. GW200311_115853-v1
3. GW200308_173609-v1
4. GW200306_093714-v1
5. GW200302_015811-v1
6. GW200225_060421-v1
7. GW200224_222234-v1
8. GW200220_124850-v1
9. GW200220_061928-v1
10. GW200219_094415-v1
11. GW200216_220804-v1
12. GW200210_092254-v1
13. GW200209_085452-v1
14. GW200208_222617-v1
15. GW200208_130117-v1
16. GW200202_154313-v1
17. GW200129_065458-v1
18. GW200128_022011-v1
19. GW200115_042309-v2
20. GW200112_155838-v1
21. GW191230_180458-v1
22. GW191222_033537-v1
23. GW191219_163120-v1
24. GW191216_213338-v1
25. GW191215_223052-v1
26. GW191204_171526-v1
27. GW191204_110529-v1
28. GW191129_134029-v1
29. GW191127_050227-v1
30. GW191126_115259-v1
31. GW191113_071753-v1
32. GW191109_010717-v1
33. GW191105_143521-v1
34. GW191103_012549-v1
35. GW190930_133541-v2
36. GW190929_012149-v2
37. GW190926_050336-v1
38. GW190925_232845-v1
39. GW190924_021846-v2
40. GW190917_114630-v1
41. GW190916_200658-v1
42. GW190915_235702-v2
43. GW190910_112807-v

In [4]:
def contains_pattern(s, options_list):
    combined_pattern = "|".join(options_list)
    return bool(re.search(combined_pattern, s))

In [5]:
N_events = 0
ignore_list = []

for i in tqdm(range(len(events))):
    event = events[i]
    # read file with name event
    event_file = f"snapshot-2023-11-04/GWTC/{event}.json"
    event_df = pd.read_json(event_file)
    
    key_val = list(event_df["events"][f"{event}"]["parameters"].keys())[0]
    
    if contains_pattern(event, ["GW190925", "GW190917", "GW190805", "GW190725"]):
        key_val = list(event_df["events"][f"{event}"]["parameters"].keys())[1]
    
    url = event_df["events"][f"{event}"]["parameters"][key_val]["data_url"]
    
    if event_df["events"][f"{event}"]["far"] > 0.25:
        ignore_list.append(event)
        continue
    
    if "https" not in url:
        for key_val in event_df["events"][f"{event}"]["parameters"].keys():
            url = event_df["events"][f"{event}"]["parameters"][key_val]["data_url"]
            if "https" in url:
                break
    
    
    pattern1= r'([^/]+)/content$'
    pattern2 = r'(?<=/)([^/]+\.h5)'
    pattern3 = r'(?<=/)([^/]+\.hdf5)'
    match1 = re.search(pattern1, url)
    match2 = re.search(pattern2, url)
    match3 = re.search(pattern3, url)
    
    if match1:
        filename = match1.group(1)
    elif match2:
        filename = match2.group(1)
    elif match3:
        filename = match3.group(1)
    else:
        print(f"No match found for {event} at i = {i}")
        break
    
    file_path = f"../{filename}"  # Change the filename as needed
    
    N_events += 1
    
    if os.path.exists(file_path):
        continue
    
    with open(file_path, "wb") as file:  # Use 'wb' to write binary content
        file.write(requests.get(url, allow_redirects=True).content)
print(f"Downloaded {N_events} events.")

100%|██████████| 93/93 [00:00<00:00, 1528.02it/s]

Downloaded 66 events.


In [9]:
# Get events of interest: GW170817, GW190425, GW190814, GW200105, GW200115, GW230529
get_list = ["GW170817", "GW190425", "GW190814", "GW200105", "GW200115", "GW230529", "GW190917"]

N_events = 0
ignore_list = []

for i in tqdm(range(len(events))):
    event = events[i]
    
    if not contains_pattern(event, get_list):
        continue
    
    # read file with name event
    event_file = f"snapshot-2023-11-04/GWTC/{event}.json"
    event_df = pd.read_json(event_file)
    
    key_val = list(event_df["events"][f"{event}"]["parameters"].keys())[0]
    
    if contains_pattern(event, ["GW190925", "GW190917", "GW190805", "GW190725"]):
        key_val = list(event_df["events"][f"{event}"]["parameters"].keys())[1]
    
    url = event_df["events"][f"{event}"]["parameters"][key_val]["data_url"]
    
    if "https" not in url:
        for key_val in event_df["events"][f"{event}"]["parameters"].keys():
            url = event_df["events"][f"{event}"]["parameters"][key_val]["data_url"]
            if "https" in url:
                break
   
    pattern1= r'([^/]+)/content$'
    pattern2 = r'(?<=/)([^/]+\.h5)'
    pattern3 = r'(?<=/)([^/]+\.hdf5)'
    match1 = re.search(pattern1, url)
    match2 = re.search(pattern2, url)
    match3 = re.search(pattern3, url)
    
    if match1:
        filename = match1.group(1)
    elif match2:
        filename = match2.group(1)
    elif match3:
        filename = match3.group(1)
    else:
        print(f"No match found for {event} at i = {i}")
        break
    
    file_path = f"../../events_of_interest/{filename}"  # Change the filename as needed
    
    N_events += 1
    
    if os.path.exists(file_path):
        continue
    
    with open(file_path, "wb") as file:  # Use 'wb' to write binary content
        file.write(requests.get(url, allow_redirects=True).content)
print(f"Downloaded {N_events} events of interest.")

100%|██████████| 93/93 [00:00<00:00, 15791.68it/s]

Downloaded 5 events of interest.


In [10]:
# Manually downloading GW230529 and GW200105
# GW230529 - https://gwosc.org/eventapi/html/O4_Discovery_Papers/GW230529_181500/v1/
# GW200105 - https://gwosc.org/eventapi/html/O3_Discovery_Papers/GW200105_162426/v1/
manual_links = ["https://dcc.ligo.org/public/0175/P2100143/002/GW200105_162426_posterior_samples_v2.h5", "https://zenodo.org/api/records/10845779/files/posterior_samples.h5/content"]
names_list = ["GW200105.h5", "GW230529.h5"]
types_list = ["GWTC3-marginal", "GWTC4"]
N_events = 0

for url, name, typ in list(zip(manual_links, names_list, types_list)):
    
    file_save_name = "IGWN-" + typ + "-manual-download" "-" + name
    file_path = f"../../events_of_interest/{file_save_name}"
    N_events += 1
     
    if os.path.exists(file_path):
        continue
    
    with open(file_path, "wb") as file:
        file.write(requests.get(url, allow_redirects=True).content)

print(f"Downloaded {N_events} other events of interest.")

Downloaded 2 other events of interest.
